In [164]:
import numpy as np
import pandas as pd
from pandas import read_csv, DataFrame, Series
import sklearn 
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
import pylab as pl
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
import pylab as pl
import matplotlib as plt


In [165]:
article_read = pd.read_csv(
    'marketing_campaign_train.csv', 
    delimiter=';'
)

In [166]:
article_read = article_read.drop(['кредит_дефолт','контакт_период','контакт_прошлый_количество','контакт_прошлый_исход', 'Id.1'],axis=1) 

In [167]:
article_read

,Id,возраст,профессия,семейное_положение,образование,жилищный_кредит,кредит,взаимодействие,контакт_месяц,контакт_день_недели,контакт_количество,контакт_исход
0,0,33,администриратор,не женат/не замужем,специальное,0,0,мобильный,5,пн,1,1
1,1,33,техник/механик,не женат/не замужем,специальное,1,0,мобильный,8,ср,1,0
2,2,30,администриратор,не женат/не замужем,высшее,1,0,мобильный,5,пн,1,0
3,3,31,администриратор,женат/замужем,высшее,1,0,домашний,5,пт,2,0
4,4,31,сфера услуг,не женат/не замужем,среднее,1,0,мобильный,5,чт,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
31183,31183,24,администриратор,не женат/не замужем,высшее,1,0,домашний,4,вт,1,0
31184,31184,38,техник/механик,женат/замужем,специальное,0,0,мобильный,3,вт,3,0
31185,31185,52,администриратор,женат/замужем,среднее,1,0,мобильный,11,пт,2,0
31186,31186,24,техник/механик,не женат/не замужем,среднее,0,0,мобильный,7,пт,2,0


In [168]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
dicts = {}

label.fit(article_read.профессия.drop_duplicates()) 
dicts['профессия'] = list(label.classes_)
article_read.профессия = label.transform(article_read.профессия) 

label.fit(article_read.семейное_положение.drop_duplicates())
dicts['семейное_положение'] = list(label.classes_)
article_read.семейное_положение = label.transform(article_read.семейное_положение)

label.fit(article_read.образование.drop_duplicates())
dicts['образование'] = list(label.classes_)
article_read.образование = label.transform(article_read.образование)

label.fit(article_read.взаимодействие.drop_duplicates())
dicts['взаимодействие'] = list(label.classes_)
article_read.взаимодействие = label.transform(article_read.взаимодействие)

label.fit(article_read.контакт_день_недели.drop_duplicates())
dicts['контакт_день_недели'] = list(label.classes_)
article_read.контакт_день_недели = label.transform(article_read.контакт_день_недели)

In [169]:
article_read

,Id,возраст,профессия,семейное_положение,образование,жилищный_кредит,кредит,взаимодействие,контакт_месяц,контакт_день_недели,контакт_количество,контакт_исход
0,0,33,0,1,3,0,0,1,5,1,1,1
1,1,33,9,1,3,1,0,1,8,3,1,0
2,2,30,0,1,1,1,0,1,5,1,1,0
3,3,31,0,0,1,1,0,0,5,2,2,0
4,4,31,8,1,4,1,0,1,5,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
31183,31183,24,0,1,1,1,0,0,4,0,1,0
31184,31184,38,9,0,3,0,0,1,3,0,3,0
31185,31185,52,0,0,4,1,0,1,11,2,2,0
31186,31186,24,9,1,4,0,0,1,7,2,2,0


In [170]:
target = article_read.контакт_исход
train = article_read.drop(['контакт_исход'], axis=1)

ROCtrainTRN, ROCtestTRN, ROCtrainTRG, ROCtestTRG = train_test_split(train, article_read.контакт_исход, test_size=0.25) 

In [171]:
model_rfc = RandomForestClassifier(n_estimators = 70) 
model_knc = KNeighborsClassifier(n_neighbors = 18) 
model_lr = LogisticRegression(penalty='l1', tol=0.01) 
model_svc = svm.SVC() 

In [172]:
test = pd.read_csv(
    'marketing_campaign_test.csv', 
    delimiter=';'
)
test = test.drop(['кредит_дефолт','контакт_период','контакт_прошлый_количество','контакт_прошлый_исход'],axis=1) 

from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
dicts = {}

label.fit(test.профессия.drop_duplicates()) 
dicts['профессия'] = list(label.classes_)
test.профессия = label.transform(test.профессия) 

label.fit(test.семейное_положение.drop_duplicates())
dicts['семейное_положение'] = list(label.classes_)
test.семейное_положение = label.transform(test.семейное_положение)

label.fit(test.образование.drop_duplicates())
dicts['образование'] = list(label.classes_)
test.образование = label.transform(test.образование)

label.fit(test.взаимодействие.drop_duplicates())
dicts['взаимодействие'] = list(label.classes_)
test.взаимодействие = label.transform(test.взаимодействие)

label.fit(test.контакт_день_недели.drop_duplicates())
dicts['контакт_день_недели'] = list(label.classes_)
test.контакт_день_недели = label.transform(test.контакт_день_недели)

In [173]:
test

,Id,возраст,профессия,семейное_положение,образование,жилищный_кредит,кредит,взаимодействие,контакт_месяц,контакт_день_недели,контакт_количество
0,31188,30,8,0,2,1,0,1,11,1,1
1,31189,30,9,1,3,1,0,1,8,0,12
2,31190,36,6,0,4,0,0,0,5,4,1
3,31191,47,9,0,4,0,0,1,4,1,1
4,31192,64,3,0,1,0,0,1,6,4,1
...,...,...,...,...,...,...,...,...,...,...,...
9995,41183,42,8,0,4,1,0,0,5,4,2
9996,41184,29,0,1,1,0,0,1,8,3,2
9997,41185,30,0,0,1,0,0,1,4,0,3
9998,41186,33,1,1,1,0,0,1,8,1,1


In [180]:
model_rfc.fit(train, target)
result = DataFrame()
result.insert(0,'Out', model_rfc.predict(test))
result.to_csv('D:/bank/test.csv', index=False)